In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import PolynomialFeatures,scale
from sklearn.preprocessing import Imputer

import pymc3 as pm
#import seaborn as sns; sns.set()
from scipy import stats, optimize
from sklearn.datasets import load_diabetes
from sklearn.cross_validation import train_test_split
from theano import shared
import theano.tensor as T
from pymc3 import *
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Importing dataset
df = pd.read_csv('breast-cancer-wisconsin.csv')
#df.dropna(how='any',inplace=True)

df.drop(['id'],1,inplace=True)
#df.replace('?',-99999,inplace=True)

##IMP NOTE: I basically dropped 'bare_nuclei' bcuz of 16 "?" values
X = scale(np.array(df.drop(['class','bare_nuclei'],1)))
y = np.array(df['class'])

#Split Data
X_tr, X_te, y_tr, y_te = train_test_split(X,y,test_size=0.2, random_state=42)

#Preprocess data for Modeling
ann_input = shared(X_tr)
ann_output = shared(y_tr)
n_hidden = 5

In [6]:
init_1 = np.random.randn(X.shape[1], n_hidden)
init_2 = np.random.randn(n_hidden, n_hidden)
init_out = np.random.randn(n_hidden)

with pm.Model() as neural_network:
    # Weights from input to hidden layer
    weights_in_1 = pm.Normal('w_in_1', 0, sd=1,
    shape=(X.shape[1], n_hidden),
    testval=init_1)

    # Weights from 1st to 2nd layer
    weights_1_2 = pm.Normal('w_1_2', 0, sd=1,
                        shape=(n_hidden, n_hidden),
                        testval=init_2)

    # Weights from hidden layer to output
    weights_2_out = pm.Normal('w_2_out', 0, sd=1,
                          shape=(n_hidden,),
                          testval=init_out)

    # Build neural-network
    act_1 = T.tanh(T.dot(ann_input, weights_in_1))
    act_2 = T.tanh(T.dot(act_1, weights_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, weights_2_out))

    out = pm.Bernoulli('data',
                   act_out,
                   observed=ann_output)

In [7]:
#infering parameters
with neural_network:
    inference=pm.ADVI()
    approx = pm.fit(n=5000,more_replacements={
        ann_input:pm.Minibatch(X_tr),
        ann_output:pm.Minibatch(y_tr)
    
     }
        )

Average Loss = inf: 100%|██████████████████████████████████████████████████████████| 5000/5000 [07:58<00:00, 10.45it/s]
Finished [100%]: Average Loss = nan


In [8]:
#Replace shared variables with testing set
#(note that using this trick we could be streaming ADVI for big data)
ann_input.set_value(X_te)
ann_output.set_value(y_te)

In [12]:
#Creater posterior predictive samples
trace= approx.sample(draws=5000)
ppc = pm.sample_ppc(trace, model=neural_network, samples=500)
pred = ppc['data'].mean(axis=0) > 0.5

print('Accuracy = {}%'.format((y_te == pred).mean() * 100))

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 702.55it/s]


Accuracy = 0.0%
